---
title: 다양한 모델로 살펴보는 캘리포니아 집값 예측(회귀)
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import koreanize_matplotlib

from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

## 데이터 읽어오기

In [ ]:
data = pd.read_csv('data/housing.csv')
data.head()

In [ ]:
data.columns.values

In [ ]:
data.info()

## 전처리(NaN, Missing Value)

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

- 한 블록 내의 최대 침실이 6445개이고 평균 침실이 537임을 알 수 있습니다. 
- 데이터가 왜곡된 것 같으므로 히스토그램을 통해 이를 확인하겠습니다.

In [ ]:
plt.figure(figsize= (10, 6))
sns.histplot(data['total_bedrooms'], color = '#005b96', kde= True);

- 확실히 왜곡되어 있으므로 누락된 값은 블록 내 객실 수 중앙값으로 채웁니다.

In [ ]:
data['total_bedrooms'] = data['total_bedrooms'].fillna(data['total_bedrooms'].median())

## EDA

In [ ]:
plt.figure(figsize= (20, 8))
sns.heatmap(data.corr(numeric_only=True), annot= True, cmap='YlGnBu')
plt.show()

- 중위소득은 분명 가장 중요한 특징입니다.

In [ ]:
sns.histplot(data['median_house_value'], color = '#005b96', kde= True);

In [ ]:
data['median_house_value'].skew()

- 우리의 목표 변수는 분명히 왜곡되어 있습니다. 따라서 로그 변환을 늦게 적용할 것입니다.

In [ ]:
data.hist(bins = 30, figsize=(20, 15), color = '#005b96');

많은 기능이 왜곡되어 있다는 것을 분명히 알 수 있습니다. 따라서 나중에 기능 변환을 수행할 때 이 문제를 해결해야 할 것입니다.

In [ ]:
grid = sns.PairGrid(data, vars=['total_rooms', 'housing_median_age', 'median_income', 'median_house_value'],
                    height=2, aspect = 2)
grid = grid.map_diag(plt.hist)
grid = grid.map_lower(sns.regplot, scatter_kws = {'s': 15, 'alpha': 0.7, 'color': '#005b96'}, 
                      line_kws = {'color':'orange', 'linewidth': 2})
grid = grid.map_upper(sns.kdeplot, n_levels = 10, cmap= 'coolwarm', fill = True)
plt.show()

어떻게 들여다봐도 문제가 많은 데이터..?, 범주형 변수도 같이 확인해보죠

In [ ]:
sns.countplot(x = data['ocean_proximity']);

In [ ]:
data.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
        s=data["population"]/100, label="population", figsize=(15,8),
        c="median_house_value", cmap=plt.get_cmap("jet"),colorbar=True,
    )
plt.legend()
plt.show()

### 특성공학

In [ ]:
data['bed_per_room'] = data['total_bedrooms'] / data['total_rooms']

In [ ]:
X = data.drop(['median_house_value'], axis=1)
y = np.log(data.median_house_value) # 로그 변환

## 특성 변환

In [ ]:
skew_df = pd.DataFrame(X.select_dtypes(np.number).columns, columns= ['Feature'])
skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: skew(X[feature]))
skew_df['Abs_Skew'] = skew_df['Skew'].apply(abs)
skew_df['Skewed'] = skew_df['Abs_Skew'].apply(lambda x: True if x > 0.5 else False)
skew_df

In [ ]:
skewed_columns = skew_df[skew_df['Abs_Skew'] > 0.5]['Feature'].values
skewed_columns

In [ ]:
for column in skewed_columns:
    X[column] = np.log(X[column])

### Encoding

In [ ]:
encoder=LabelEncoder()
X['ocean_proximity']=encoder.fit_transform(X['ocean_proximity'])

### Scaling

In [ ]:
X.head()

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index= X.index, columns= X.columns)


## 데이터 나누기

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

## 모델 생성

### Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

### KNN

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
predictions_knn = knn.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_knn))
r2 = r2_score(y_test, predictions_knn)

print('RMSE:', rmse)
print('R-square:', r2)

### Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators= 100)
rf.fit(X_train, y_train)
predictions_rf = rf.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_rf))
r2 = r2_score(y_test, predictions_rf)

print('RMSE:', rmse)
print('R-square:', r2)

### CatBoost

In [ ]:
catboost = CatBoostRegressor(verbose= 0)
catboost.fit(X_train, y_train)
predictions_cb = catboost.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_cb))
r2 = r2_score(y_test, predictions_cb)

print('RMSE:', rmse)
print('R-square:', r2)

### XGBoost

In [ ]:
xgboost = XGBRegressor()
xgboost.fit(X_train, y_train)
predictions_xgb = xgboost.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_xgb))
r2 = r2_score(y_test, predictions_xgb)

print('RMSE:', rmse)
print('R-square:', r2)

### LightGBM

In [ ]:
lgb = LGBMRegressor()
lgb.fit(X_train, y_train)
predictions_lgb = lgb.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_lgb))
r2 = r2_score(y_test, predictions_lgb)

print('RMSE:', rmse)
print('R-square:', r2)

### Gradient Boosting

In [ ]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
predictions_gbr = gbr.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions_gbr))
r2 = r2_score(y_test, predictions_gbr)

print('RMSE:', rmse)
print('R-square:', r2)

## 모델 결정

CatBoost, XGBoost, LightGBM, RandomForest 성능은 다중 선형 회귀, knn, Gradient Boosting 능가하는 것으로 나타났습니다. 이제 이 네 가지 모델을 결합하여 최종 예측을 해보겠습니다.

In [ ]:
final_predictions = (
    0.25 * predictions_cb+
    0.25 * predictions_rf+
    0.25 * predictions_xgb+
    0.25 * predictions_lgb
)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
r2 = r2_score(y_test, final_predictions)

print('RMSE:', rmse)
print('R-square:', r2)

In [ ]:
final_predictions

최종 예측을 원래 규모로 되돌리려면 최종 예측의 지수를 취해야 합니다.

In [ ]:
final_predictions = np.exp(final_predictions)
y_test = np.exp(y_test)

In [ ]:
pd.DataFrame({'Actual': y_test, 'Predicted': final_predictions.round(2)})

## 결과 확인

In [ ]:
plt.figure(figsize= (10, 6))
sns.scatterplot(x= y_test, y= final_predictions, color= '#005b96')
plt.xlabel('Actual House value')
plt.ylabel('Predicted House Value')
plt.show()

In [ ]:
plt.figure(figsize= (10, 6))
sns.residplot(x= y_test, y = final_predictions, color= '#005b96')
plt.show()

In [ ]:
resid = y_test - final_predictions
plt.figure(figsize= (10, 6))
sns.histplot(resid)
plt.xlabel('Error');

오류의 분포가 정상적으로 보이기 때문에 우리 모델이 제대로 작동하고 있는 것입니다.